<a href="https://colab.research.google.com/github/YoshiyukiKono/gen_ai-sandbox/blob/main/langchain_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/1LS7ag0jZlUxEdgCi1NG9S90cfTkCBQbH?usp=sharing#scrollTo=ydyK_bgrOHbG

In [1]:
!pip install openai==0.27.9
!pip install langchain==0.0.272
!pip install tiktoken==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.6 MB/s eta 0:00:00


In [ ]:
import os
import openai

#TODO: APIキーの登録が必要
os.environ["OPENAI_API_KEY"] = "..."
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
%%writefile train.jsonl
{"messages": [{"role": "user", "content": "Fine-tuningって何？"}, {"role": "assistant", "content": "ご質問ありがとうございます！Fine-tuningとは、学習済みの言語モデルを特定のタスクに最適化することです。"}]}
{"messages": [{"role": "user", "content": "Fine-tuningのメリットを3つ教えて。"}, {"role": "assistant", "content": "ご質問ありがとうございます！「参考テキストの入力の手間が省ける点」と「膨大な知識量を学習できる点」と「システムとしての応答速度が改善される点」が挙げられます。"}]}
{"messages": [{"role": "user", "content": "Fine-tuningのデメリットを3つ教えて。"}, {"role": "assistant", "content": "ご質問ありがとうございます！「学習コストがかかる点」と「利用コストが高くなる点」と「AIとしての汎用性は下がってしまう点」が挙げられます。"}]}
{"messages": [{"role": "user", "content": "LangChainとは？"}, {"role": "assistant", "content": "ご質問ありがとうございます！LangChainとは「ChatGPTなどの言語系モデルの機能拡張を効率的に実装するためのライブラリ」です。"}]}
{"messages": [{"role": "user", "content": "LangChainのメリットは何？"}, {"role": "assistant", "content": "ご質問ありがとうございます！LangChainを用いることで、最新情報に対応できる点や長文を入力できる点、複雑な計算問題に回答できる点が挙げられます。"}]}
{"messages": [{"role": "user", "content": "LangChainの実装方法を教えて。"}, {"role": "assistant", "content": "ご質問ありがとうございます！LangChainを使用するための環境構築やAPIキーの設定方法、さらに具体的なコード例を紹介します。まずは必要なライブラリのインストールとAPIキーの設定を行い、その後OpenAIモデルの呼び出しやプロンプトのテンプレート作成、LLM Chainの実行方法を示します。"}]}
{"messages": [{"role": "user", "content": "LangChainの具体的な機能は何？"}, {"role": "assistant", "content": "ご質問ありがとうございます！LangChainは複数の具体的な機能を実装しています。具体的な機能としては、Models（言語モデルの切り替えや組み合わせ）、Prompt（プロンプトの管理や最適化）、Indexes（外部データを用いた回答生成）、Chains（複数のプロンプト入力の実行）、Agents（モデルによるアクションの決定と実行）、Memory（状態保持）などがあります。"}]}
{"messages": [{"role": "user", "content": "LangChainのAgents機能はどのようなもの？"}, {"role": "assistant", "content": "ご質問ありがとうございます！Agentsは言語モデルにツールを用いて次のアクションを決定し、実行、観測、完了する機能です。例えば、与えられたツールに基づいてモデルがGoogle検索をし、Pythonコードを生成し、テストを行ってくれるイメージです。"}]}
{"messages": [{"role": "user", "content": "LangChainのIndexes機能はどのような場面で役立つ？"}, {"role": "assistant", "content": "ご質問ありがとうございます！IndexesはPDFやCSVなどの外部データを用いて回答を生成する機能です。例えば、社内のPDFを利用してチャットボットを作成する場面で便利です。また、機密情報を外部サーバーに送信せずに自社システム上で効率的に実装できる点も魅力です。"}]}
{"messages": [{"role": "user", "content": "LangChainのMemory機能は何に役立つ？"}, {"role": "assistant", "content": "ご質問ありがとうございます！MemoryはChainsやAgents内部の状態保持を可能にする機能です。これにより、チャットのやり取りの状態を保持し、過去の会話を考慮した回答を生成できるようになります。従来の静的なクエリ単位の処理から進化した機能と言えるでしょう。"}]}

Writing train.jsonl


In [5]:
!cat train.jsonl

{"messages": [{"role": "user", "content": "Fine-tuningって何？"}, {"role": "assistant", "content": "ご質問ありがとうございます！Fine-tuningとは、学習済みの言語モデルを特定のタスクに最適化することです。"}]}
{"messages": [{"role": "user", "content": "Fine-tuningのメリットを3つ教えて。"}, {"role": "assistant", "content": "ご質問ありがとうございます！「参考テキストの入力の手間が省ける点」と「膨大な知識量を学習できる点」と「システムとしての応答速度が改善される点」が挙げられます。"}]}
{"messages": [{"role": "user", "content": "Fine-tuningのデメリットを3つ教えて。"}, {"role": "assistant", "content": "ご質問ありがとうございます！「学習コストがかかる点」と「利用コストが高くなる点」と「AIとしての汎用性は下がってしまう点」が挙げられます。"}]}
{"messages": [{"role": "user", "content": "LangChainとは？"}, {"role": "assistant", "content": "ご質問ありがとうございます！LangChainとは「ChatGPTなどの言語系モデルの機能拡張を効率的に実装するためのライブラリ」です。"}]}
{"messages": [{"role": "user", "content": "LangChainのメリットは何？"}, {"role": "assistant", "content": "ご質問ありがとうございます！LangChainを用いることで、最新情報に対応できる点や長文を入力できる点、複雑な計算問題に回答できる点が挙げられます。"}]}
{"messages": [{"role": "user", "content": "LangChainの実装方法を教えて。"}, {"role": "assistant", "content": "ご質問ありがとうございます！LangChainを使用するための環

In [ ]:
import json
import os
import tiktoken
import numpy as np
from collections import defaultdict

with open(jsonl_file_path) as f:
    dataset = [json.loads(line) for line in f]

print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(
            message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

encoding = tiktoken.get_encoding("cl100k_base")


# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(
    messages,
    tokens_per_message=3,
    tokens_per_name=1,
):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens


def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens


def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(
    f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning"
)

MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(
    min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(
    f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training"
)
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(
    f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens"
)
print("See pricing page to estimate total costs")

In [ ]:
res = openai.File.create(
    file=open(jsonl_file_path, "rb"),
    purpose="fine-tune",
)

file_id = res["id"]
print(file_id)

In [ ]:
res = openai.FineTuningJob.create(training_file=file_id, model="gpt-3.5-turbo")
ft_id = res["id"]
print(ft_id)

In [ ]:
# 最大10個のモデルを取得
model_list = openai.FineTuningJob.list(limit=10)
print(model_list)

# # モデルの状態を取得
# openai.FineTuningJob.retrieve(ft_id)

# # 特定のモデルにおける最大10個のジョブイベントを出力
# openai.FineTuningJob.list_events(id=ft_id, limit=10)

In [ ]:
fine_tuned_model_name = model_list["data"][0]["fine_tuned_model"]
print(fine_tuned_model_name)

In [ ]:
# # ジョブのキャンセル
# openai.FineTuningJob.cancel(ft_id)

In [ ]:
# # モデルの削除（組織のオーナーのみ実行可能）
# openai.Model.delete(ft_id)

In [ ]:
completion = openai.ChatCompletion.create(
    model=fine_tuned_model_name,
    messages=[{
        "role": "user",
        "content": "ChatGPTについて教えてください。"
    }],
)

print(completion.choices[0].message["content"])

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model=fine_tuned_model_name)
res = llm.predict("ChatGPTについて教えてください。")
print(res)